In [14]:
import re

log_entries = []

# Read the log file
with open('data/haproxytmp.log', 'r') as file:
    log_entries = file.readlines()

# Define regular expression for field extraction
regex_pattern = r'^(?P<timestamp>\w+\s+\d+\s\d+:\d+:\d+)\s+(?P<server>\S+)\s+haproxy\[\d+\]:\s(?P<client_ip>[\d.:]+):(?P<client_port>\d+)\s+\[(?P<datetime>[^\]]+)\]\s+(?P<frontend>\S+)~\s+(?P<backend>[^\s/]+)/(?P<server_name>[^\s]+)'

fields = []

# Iterate through log entries
for log_entry in log_entries:
    # Extract fields using regular expression
    match = re.match(regex_pattern, log_entry)
    if match:
        fields.append({
            'timestamp': match.group('timestamp'),
            'server': match.group('server'),
            'client_ip': match.group('client_ip'),
            'client_port': match.group('client_port'),
            'datetime': match.group('datetime'),
            'frontend': match.group('frontend'),
            'backend': match.group('backend')
        })

# Print extracted fields
for field in fields:
    print(field)


{'timestamp': 'Jun  6 07:35:12', 'server': 'interswitch-virtual-machine', 'client_ip': '41.210.186.148', 'client_port': '51732', 'datetime': '06/Jun/2023:07:35:12.147', 'frontend': 'www-https', 'backend': 'interswitchstore_backend'}
{'timestamp': 'Jun  6 07:35:12', 'server': 'interswitch-virtual-machine', 'client_ip': '196.13.203.201', 'client_port': '59225', 'datetime': '06/Jun/2023:07:35:12.378', 'frontend': 'www-https', 'backend': 'api_backend'}
{'timestamp': 'Jun  6 07:35:12', 'server': 'interswitch-virtual-machine', 'client_ip': '197.239.10.226', 'client_port': '1517', 'datetime': '06/Jun/2023:07:35:12.612', 'frontend': 'www-https', 'backend': 'interswitchstore_backend'}
{'timestamp': 'Jun  6 07:35:12', 'server': 'interswitch-virtual-machine', 'client_ip': '196.13.203.201', 'client_port': '59225', 'datetime': '06/Jun/2023:07:35:12.569', 'frontend': 'www-https', 'backend': 'api_backend'}
{'timestamp': 'Jun  6 07:35:13', 'server': 'interswitch-virtual-machine', 'client_ip': '196.13.